### SGD 방식 구현

* 랜덤하게 데이터를 섞기 위한 함수

  * torch.randperm(n) : 0 ~ n -1 까지의 정수를 랜덤하게 섞어서, 순열(배열)을 만들어 줌
  * torch.index_select(텐서 객체, 차원번호, 인덱스 텐서)

    * 차원번호는 예를 들어, |x| = (3,4)에서 0차원에 해당하는 값은 3 (행으로 이해하면 됨), 1차원에 해당하는 값은 4 (열로 이해하면 됨)
  
  * 특정 차원의 나열된 인덱스 번호 순서대로, 데이터를 섞어줌

In [ ]:
import torch

data1 = torch.randn(3, 4)
print(data1)
indices = torch.tensor([1, 2])
print(indices)
print(torch.index_select(data1, 0, indices))

tensor([[-0.3109,  2.4433,  1.1169, -0.5429],
        [ 2.7027, -0.5614, -1.0916,  0.7503],
        [-0.4048, -0.4643,  0.5706, -0.8950]])
tensor([1, 2])
tensor([[ 2.7027, -0.5614, -1.0916,  0.7503],
        [-0.4048, -0.4643,  0.5706, -0.8950]])
